## MovieLens 20M Dataset Recommender System - 01_Data Wrangling and EDA

This dataset describes 5-star rating and free-text tagging activity from [MovieLens](http://movielens.org), a movie recommendation service. These data were created by users between January 09, 1995 and March 31, 2015. This dataset was generated on March 31, 2015, and updated on October 17, 2016 to update links.csv and add genome-* files. 

Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided,

The data are contained in six files, 'genome-scores.csv', 'genome-tags.csv', 'links.csv', 'movies.csv', 'ratings.csv' and 'tags.csv'.

We would like to use it to build a model of recommender systems.

This is the first part of the study: Data Wrangling and Exploratory Data Analysis (EDA).

### Content
### 1. Import Modules and Read Data Files
#### 1.1 Import  Necessary Modules
#### 1.2 Read Movie File
#### 1.3 Read Rating File
#### 1.4 Read Tag File
#### 1.5 Read Genome Score File, Genome Tag File, and Links File
### 2. Explore the data
### 3. Data Wrangling and Export the Data

### 1. Import Modules and Read Data Files

#### 1.1 Import  Necessary Modules

In [1]:
import pandas as pd
import numpy as np

#### 1.2 Read Movie File

In [2]:
# First, read the movie file.
movies = pd.read_csv('./ml-20m/movies.csv')

In [3]:
# look at the first few rows of the movie file.
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
# look at the size of the movie file.
movies.shape

(27278, 3)

In [5]:
# see if every movie is unique.
movies.movieId.nunique()

27278

We can see that every row is one movie and they are all unique in the movie file.
Let's then read the rating file.

#### 1.3 Read Rating File

In [6]:
# read the rating file.
ratings = pd.read_csv('./ml-20m/ratings.csv')

In [7]:
# look at first few rows in the rating file.
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


We are curious to see if the rating time is related the rating values. So let's extract weekday, month and year from the timestamp, and see if the average rating differs among months, years, etc.  

In [8]:
# convert timestamp to datetime
ratings['rating_time'] = pd.to_datetime(ratings['timestamp'], unit='s')

In [9]:
ratings.head()

,userId,movieId,rating,timestamp,rating_time
0,1,2,3.5,1112486027,2005-04-02 23:53:47
1,1,29,3.5,1112484676,2005-04-02 23:31:16
2,1,32,3.5,1112484819,2005-04-02 23:33:39
3,1,47,3.5,1112484727,2005-04-02 23:32:07
4,1,50,3.5,1112484580,2005-04-02 23:29:40


In [10]:
# extract day of week
ratings['rating_weekday'] = ratings['rating_time'].dt.dayofweek

In [11]:
# extract month
ratings['rating_month'] = ratings['rating_time'].dt.month

In [12]:
# extract year
ratings['rating_year'] = ratings['rating_time'].dt.year

In [13]:
ratings.head()

,userId,movieId,rating,timestamp,rating_time,rating_weekday,rating_month,rating_year
0,1,2,3.5,1112486027,2005-04-02 23:53:47,5,4,2005
1,1,29,3.5,1112484676,2005-04-02 23:31:16,5,4,2005
2,1,32,3.5,1112484819,2005-04-02 23:33:39,5,4,2005
3,1,47,3.5,1112484727,2005-04-02 23:32:07,5,4,2005
4,1,50,3.5,1112484580,2005-04-02 23:29:40,5,4,2005


In [14]:
# get the average rating at different weekdays
ratings.groupby('rating_weekday')['rating'].mean()

rating_weekday
0    3.522597
1    3.520359
2    3.515987
3    3.519239
4    3.526774
5    3.541368
6    3.533937
Name: rating, dtype: float64

In [15]:
# get the average rating at different months
ratings.groupby('rating_month')['rating'].mean()

rating_month
1     3.520437
2     3.524092
3     3.496393
4     3.528203
5     3.514545
6     3.513506
7     3.517270
8     3.505279
9     3.528368
10    3.565256
11    3.546288
12    3.539704
Name: rating, dtype: float64

In [16]:
# get the average rating at different year
ratings.groupby('rating_year')['rating'].mean()

rating_year
1995    3.750000
1996    3.545408
1997    3.588522
1998    3.512484
1999    3.616337
2000    3.576119
2001    3.533771
2002    3.482654
2003    3.474775
2004    3.429614
2005    3.434001
2006    3.463172
2007    3.471741
2008    3.539019
2009    3.508644
2010    3.533408
2011    3.562192
2012    3.609827
2013    3.644767
2014    3.607711
2015    3.499819
Name: rating, dtype: float64

In [17]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 8 columns):
 #   Column          Dtype         
---  ------          -----         
 0   userId          int64         
 1   movieId         int64         
 2   rating          float64       
 3   timestamp       int64         
 4   rating_time     datetime64[ns]
 5   rating_weekday  int64         
 6   rating_month    int64         
 7   rating_year     int64         
dtypes: datetime64[ns](1), float64(1), int64(6)
memory usage: 1.2 GB


We can see that the rating at different weekday and months are quite close to each other, and rating at different years are different. So we would like to keep the year and remove the remaining. 

In [18]:
# column 'timestamp' is not necessary to our model. So let's drop it.
ratings = ratings.drop(columns=['timestamp', 'rating_time', 'rating_weekday', 'rating_month'])

In [19]:
# see if there is missing values in the rating file.
ratings.isna().sum()

userId         0
movieId        0
rating         0
rating_year    0
dtype: int64

It shows that there is no missing values in the rating file.

In [20]:
# look at the size of the rating file. 
ratings.shape

(20000263, 4)

Next we want to build a model to predict users' rating. The steps are: 1. select a data with suitable size; 2. Split the data into train and test set; 3. Apply several algorithms and compare the performance. In this Notebook we will only work on the first part, which is to create a dataset with the right size. Considering the capacity for a personal computer, we think the size of 100,000 rows is good for this study. 

In [21]:
# look at how many people who give movie ratings.
ratings['userId'].nunique()

138493

In [22]:
# look at how many movies in the rating file. 
ratings['movieId'].nunique()

26744

There are 138,493 users and 26744 movies in the rating file. 

#### 1.4 Read Tag File

In [23]:
tags = pd.read_csv('./ml-20m/tags.csv')

In [24]:
# show the first few rows
tags.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,1240597180
1,65,208,dark hero,1368150078
2,65,353,dark hero,1368150079
3,65,521,noir thriller,1368149983
4,65,592,dark hero,1368150078


In [25]:
# show the top 10 tag values
tags.tag.value_counts().head(10)

sci-fi             3384
based on a book    3281
atmospheric        2917
comedy             2779
action             2657
surreal            2427
BD-R               2334
twist ending       2323
funny              2072
dystopia           1991
Name: tag, dtype: int64

In [26]:
# show the tag file size
tags.shape

(465564, 4)

In [27]:
# show how many users give tags
tags['userId'].nunique()

7801

In [28]:
#show how many movies have tags
tags['movieId'].nunique()

19545

#### 1.5 Read Genome Score File, Genome Tag File, and Links File

In [29]:
genome_scores = pd.read_csv('./ml-20m/genome-scores.csv')

In [30]:
genome_tags = pd.read_csv('./ml-20m/genome-tags.csv')

In [31]:
links = pd.read_csv('./ml-20m/links.csv')

In [32]:
genome_scores.head()

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [33]:
genome_tags.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [34]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [35]:
genome_scores.tagId.value_counts()

1       10381
750     10381
756     10381
755     10381
754     10381
        ...  
383     10381
384     10381
385     10381
386     10381
1128    10381
Name: tagId, Length: 1128, dtype: int64

In [36]:
genome_scores.movieId.nunique()

10381

In [37]:
genome_scores.shape

(11709768, 3)

### 2. Explore the data

Let's look at statistics on the data.

#### What is the average rating?

In [38]:
ratings['rating'].mean()

3.5255285642993797

The average rating for all movies is 3.5.

#### How many ratings each user gives on average?

In [39]:
ratings.shape[0] / ratings['userId'].nunique()

144.4135299257002

On average, each user gives 144 movie ratings.

#### Which movie has the most ratings?

In [40]:
ratings['movieId'].value_counts().head(1)

296    67310
Name: movieId, dtype: int64

In [41]:
movies[movies['movieId']==296]

,movieId,title,genres
293,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller


#### What is the rating matrix sparsity?

In [73]:
sparsity = len(final_ratings)/(final_ratings['movieId'].nunique()*final_ratings['userId'].nunique())

The movie "Pulp Fiction (1994)" is the most rated movie. It has 67310 ratings. 

In [74]:
print('The rating matrix sparsity is: ', sparsity)

The rating matrix sparsity is:  0.05412425137898957


### 3. Data Wrangling and Export the Data

Let's merge the rating file with the movie file, so that there will be information of title and genres in the rating file.

In [42]:
# merge ratings with movie file. 
new_ratings = ratings.merge(movies, on='movieId', how='left')

In [43]:
# check the size of the rating file before and after the merge.
ratings.shape, new_ratings.shape

((20000263, 4), (20000263, 6))

Let's then think about reducing data size. As mentioned before, our target is to reduce its size to about 100,000 rows. In order to do that, let's consider the genres, the count on users, and the count on movies.

For the genres, we wants to select only the top 20 genres.

In [44]:
# create a dataframe with the top five genres
most_rated_genres = pd.DataFrame(new_ratings.genres.value_counts(ascending=False).reset_index().head(20))

In [45]:
most_rated_genres

,index,genres
0,Drama,1467402
1,Comedy,1316161
2,Comedy|Romance,793252
3,Comedy|Drama,656474
4,Drama|Romance,644626
5,Comedy|Drama|Romance,615897
6,Crime|Drama,467417
7,Action|Adventure|Sci-Fi,441351
8,Action|Adventure|Thriller,313902
9,Action|Crime|Thriller,310685


In [46]:
most_rated_genres = most_rated_genres.rename(columns=({'index':'genres', 'genres':'count'}))

In [47]:
most_rated_genres

,genres,count
0,Drama,1467402
1,Comedy,1316161
2,Comedy|Romance,793252
3,Comedy|Drama,656474
4,Drama|Romance,644626
5,Comedy|Drama|Romance,615897
6,Crime|Drama,467417
7,Action|Adventure|Sci-Fi,441351
8,Action|Adventure|Thriller,313902
9,Action|Crime|Thriller,310685


We will then select the ratings whose genres is within the top genres

In [48]:
ratings_top_genres = new_ratings[new_ratings['genres'].isin(most_rated_genres['genres'])]

In [49]:
ratings_top_genres.shape, new_ratings.shape

((9252357, 6), (20000263, 6))

There are 400,000 rows in the new dataset. Let's then consider the count on users. We will select only users who give more than 200 ratings in the new dataset. 

In [50]:
# list total users in the new rating data.
user_counts = pd.DataFrame(ratings_top_genres.userId.value_counts().reset_index())

In [51]:
most_rating_users = user_counts[user_counts['userId']>100]

In [52]:
most_rating_users = most_rating_users.rename(columns=({'index':'userId', 'userId':'count'}))

In [53]:
most_rating_users

,userId,count
0,118205,4985
1,8405,3843
2,8963,3266
3,82418,3233
4,121535,2855
...,...,...
23635,117640,101
23636,11709,101
23637,91894,101
23638,14687,101


In [54]:
# create the new rating file
new_ratings = ratings_top_genres[ratings_top_genres['userId'].isin(most_rating_users['userId'])]

In [55]:
# check the new and old data size
new_ratings.shape, ratings_top_genres.shape

((5486765, 6), (9252357, 6))

Finally let's consider the count on movie ids in the new dataset. We will drop movies with less than 500 ratings.

In [56]:
movie_count = new_ratings['movieId'].value_counts()

In [57]:
movie_count.values

array([18386, 18377, 18336, ...,     1,     1,     1], dtype=int64)

In [58]:
most_rated_movies = movie_count[movie_count.values>100].index

In [59]:
most_rated_movies

Int64Index([  480,  2571,   318,   260,  1196,  2858,  1210,   110,   780,
              527,
            ...
            46959,  5215,  1519,  2545,  2061,   394,  6180, 62235,  6528,
             6467],
           dtype='int64', length=4078)

In [60]:
latest_ratings = new_ratings[new_ratings['movieId'].isin(most_rated_movies)]

In [61]:
latest_ratings.shape, new_ratings.shape

((5319088, 6), (5486765, 6))

In [62]:
final_users = most_rating_users.sample(frac=0.1)

In [63]:
final_users.head()

,userId,count
10519,8558,184
21642,126307,109
17866,78181,126
5190,109762,279
13491,47736,155


In [64]:
final_ratings = latest_ratings[latest_ratings['userId'].isin(final_users['userId'])]

In [65]:
final_ratings.shape, latest_ratings.shape

((521779, 6), (5319088, 6))

So the "final_ratings" is our final data for the rating predicting. Let's look at its details. 

In [66]:
final_ratings.head()

,userId,movieId,rating,rating_year,title,genres
8145,73,17,3.0,2000,Sense and Sensibility (1995),Drama|Romance
8148,73,55,5.0,2000,Georgia (1995),Drama
8149,73,110,5.0,2000,Braveheart (1995),Action|Drama|War
8150,73,111,4.0,2000,Taxi Driver (1976),Crime|Drama|Thriller
8152,73,246,4.0,2000,Hoop Dreams (1994),Documentary


Let's then count its userId, movieId, and genres.

In [67]:
final_ratings['userId'].value_counts()

125978    1960
119531    1460
128258    1368
119048    1330
112988    1323
          ... 
37660       95
105044      90
111822      87
94583       86
83024       85
Name: userId, Length: 2364, dtype: int64

In [68]:
final_ratings['movieId'].value_counts()

480       1845
318       1843
260       1828
2571      1809
1196      1781
          ... 
5090         4
26726        4
8785         4
100244       3
94931        2
Name: movieId, Length: 4078, dtype: int64

In [69]:
final_ratings['genres'].value_counts()

Drama                               87838
Comedy                              79201
Comedy|Romance                      44445
Comedy|Drama                        40086
Comedy|Drama|Romance                36596
Drama|Romance                       36566
Crime|Drama                         24270
Action|Adventure|Sci-Fi             20587
Drama|Thriller                      17405
Action|Adventure|Thriller           16268
Action|Crime|Thriller               14221
Action|Adventure|Sci-Fi|Thriller    13874
Crime|Drama|Thriller                13660
Comedy|Crime                        13403
Drama|War                           13079
Action|Sci-Fi|Thriller              11675
Action|Drama|War                    10396
Documentary                         10212
Action|Crime|Drama|Thriller          9851
Thriller                             8146
Name: genres, dtype: int64

The last step is to write the final rating data to a new csv file. 

In [72]:
file_name = './ml-20m/new_ratings.csv'
final_ratings.to_csv(file_name, index=False)